In [3]:
#@title Bibliotecas padrões
import pandas as pd
import pandas.io.sql as sqlio
import sys
import psycopg2
from ydata_profiling import ProfileReport
import numpy as np
from botocore import UNSIGNED
from botocore.config import Config
from urllib.parse import quote_plus
from pyathena import connect
import boto3
import boto3.session
from botocore import exceptions
import seaborn as sns
import plotly.express as px

#Carregando as bibliotecas necessárias para a análise estatística
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (8, 8)
import statsmodels.api as sm
import statsmodels.tsa as tsa
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import pmdarima as pm
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_process import arma_generate_sample

In [4]:
#ponte de conexão com o redshift
connection = psycopg2.connect(dbname="dw_amorsaude",
                  user="",
                  password="",
                  host ="redshift.amazonaws.com",
                  port = 5439)

In [5]:
#query
df_emails = '''select sp.id, sp.valor from stg_propostas sp'''

In [8]:
df_prontuarios = pd.read_sql_query(df_emails,con=connection)


/tmp/ipykernel_5280/1688475736.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_prontuarios = pd.read_sql_query(df_emails,con=connection)


In [13]:
# Carregando o arquivo Excel em um DataFrame
df_crm = pd.read_excel('/home/gabriel/dbt-projects/python_files/analises/files/hubspot-core-report-negocios-murph-2023-09-04-1 - Copia.xlsx')
df_crm.rename(columns={'ID Feegow': 'id'}, inplace=True)

In [20]:
 chunk_merge = pd.merge(df_crm, df_prontuarios, on='id', how='left')

In [21]:
chunk_merge.head()

,Record ID,Nome do negócio,Tabela,Valor na moeda da empresa,Unnamed: 4,Etapa do negócio,Tabulação,Proprietário do negócio,nivel de probabilidade,TIER,id,valor
0,14963003168,Pós-consulta | Claudia Barbosa Borges Cordeiro,Cartão de TODOS,265,NaN,Executada,Executada,Unidade de Grajaú,0.395347,30 a 40,15918883.0,26.5
1,14961998999,Pós-consulta | Paola Coelho Santos,Cartão de TODOS,75,NaN,Executada,Executada,campinascentro.sp@amorsaude.com,0.365344,30 a 40,15927321.0,75.0
2,14961998998,Pós-consulta | Paola Coelho Santos,Cartão de TODOS,150,NaN,Executada,Executada,campinascentro.sp@amorsaude.com,0.494073,40 a 50,15927385.0,150.0
3,14958935049,Pós-consulta | Erika Maria De Souza Reis,Cartão de TODOS,420,NaN,Executada,Executada,Unidade de Grajaú,0.657737,60 a 70,15923462.0,420.0
4,14958805676,Pós-consulta | Helena Calado Da Silva Dias,Cartão de TODOS,95,NaN,Executada,Executada,Unidade de Grajaú,0.941062,90 a 100,15921963.0,9.5


In [35]:
from openpyxl import Workbook

In [23]:
nome_do_arquivo = 'propostas_crm.xlsx'
chunk_merge.to_excel(nome_do_arquivo, index=False)  # O argumento 'index=False' evita salvar o índice do DataFrame no Excel